### Build a Simple LLM Application with LCEL
In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After seeing this video, you'll have a high level overview of:

- Using language models

- Using PromptTemplates and OutputParsers

- Using LangChain Expression Language (LCEL) to chain components together

- Debugging and tracing your application using LangSmith

- Deploying your application with LangServe

In [2]:
! pip install langchain

In [1]:
### Open AI API Key and Open Source models--Llama3,Gemma2,mistral--Groq

import os
from dotenv import load_dotenv
load_dotenv()

import openai
openai.api_key=os.getenv("OPENAI_API_KEY")

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key

'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'

In [2]:
! pip install langchain_groq

In [3]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
model=ChatGroq(model="deepseek-r1-distill-llama-70b",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x128bcd210>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x128bcde10>, model_name='deepseek-r1-distill-llama-70b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
! pip install langchain_core

In [5]:
from langchain_core.messages import HumanMessage,SystemMessage

messages=[
    SystemMessage(content="Translate the following from English to French"),
    HumanMessage(content="Hello How are you?")
]

result=model.invoke(messages)

In [6]:
result

AIMessage(content='<think>\nAlright, the user sent "Hello How are you?" in English and wants it translated to French.\n\nFirst, I\'ll break it down. "Hello" is straightforward, it\'s "Bonjour."\n\nNext, "How are you?" is a common phrase. In French, the direct translation is "Comment ça va?" but it\'s more natural to say "Comment allez-vous?" which is polite.\n\nPutting it together, the translation would be "Bonjour, comment allez-vous?"\n\nI should check if there are any nuances or if the user prefers a different form, but since it\'s a simple greeting, this should be perfect.\n</think>\n\nBonjour, comment allez-vous ?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 138, 'prompt_tokens': 16, 'total_tokens': 154, 'completion_time': 0.501818182, 'prompt_time': 0.003829565, 'queue_time': 0.234040902, 'total_time': 0.505647747}, 'model_name': 'deepseek-r1-distill-llama-70b', 'system_fingerprint': 'fp_4fc8558a04', 'finish_reason': 'stop', 'logprobs': None}, i

In [7]:
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()
parser.invoke(result)

'<think>\nAlright, the user sent "Hello How are you?" in English and wants it translated to French.\n\nFirst, I\'ll break it down. "Hello" is straightforward, it\'s "Bonjour."\n\nNext, "How are you?" is a common phrase. In French, the direct translation is "Comment ça va?" but it\'s more natural to say "Comment allez-vous?" which is polite.\n\nPutting it together, the translation would be "Bonjour, comment allez-vous?"\n\nI should check if there are any nuances or if the user prefers a different form, but since it\'s a simple greeting, this should be perfect.\n</think>\n\nBonjour, comment allez-vous ?'

In [8]:
### Using LCEL- chain the components
chain=model|parser
chain.invoke(messages)

'<think>\nAlright, the user wants to translate "Hello How are you?" into French. Let me break this down.\n\nFirst, "Hello" in French is "Bonjour". That\'s straightforward.\n\nNext, "How are you?" can be translated in a couple of ways. The most common is "Comment ça va?" which is casual and widely used.\n\nPutting it together, it would be "Bonjour, comment ça va?" to make it a complete greeting.\n\nI should also consider if the user wants a more formal version, but since the original is pretty standard, sticking with "Bonjour" and "comment ça va?" should be perfect.\n</think>\n\nBonjour, comment ça va ?'

In [9]:
### Prompt Templates
from langchain_core.prompts import ChatPromptTemplate

generic_template="Trnaslate the following into {language}:"

prompt=ChatPromptTemplate.from_messages(
    [("system",generic_template),("user","{text}")]
)



In [10]:
result=prompt.invoke({"language":"French","text":"Hello"})

In [11]:
result.to_messages()

[SystemMessage(content='Trnaslate the following into French:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]

In [12]:
##Chaining together components with LCEL
chain=prompt|model|parser
chain.invoke({"language":"French","text":"Hello"})

'<think>\n\n</think>\n\nBonjour'

In [13]:
! pip install streamlit